In [4]:
from qiskit_nature.second_q.drivers import PySCFDriver, ElectronicStructureDriver
from qiskit_nature.second_q.problems import ElectronicStructureProblem
from qiskit_nature.second_q.mappers import JordanWignerMapper, ParityMapper
from qiskit_nature.second_q.transformers import ActiveSpaceTransformer, FreezeCoreTransformer
from qiskit_nature.second_q.formats.molecule_info import MoleculeInfo
from qiskit_algorithms import VQE
from qiskit_algorithms.optimizers import SLSQP
from qiskit_nature.second_q.circuit.library.ansatzes import UCCSD
from qiskit_nature.second_q.circuit.library.initial_states import HartreeFock
from qiskit.primitives import Estimator
# https://www.webqc.org/molecularformatsconverter.php

In [10]:
with open("Ginsenosides.xyz") as f:
    list_reader = f.readlines()
xyz = list_reader[2:]

list_molecule = ';'.join([i.replace('\n', ' ') for i in xyz])
def processMolecule(xyz):
    return [i.split()[0] for i in xyz], [tuple(i.split()[1:]) for i in xyz]
molecule = MoleculeInfo(
    symbols = processMolecule(xyz)[0],
    coords = processMolecule(xyz)[1],
    charge=0
)
driver = PySCFDriver(
    atom=list_molecule,
    charge=0,
    spin=1,
)
#driver = PySCFDriver.from_molecule(driver)
problem = driver.run()


/home/codespace/.python/current/lib/python3.10/site-packages/pyscf/dft/libxc.py:771: UserWarning: Since PySCF-2.3, B3LYP (and B3P86) are changed to the VWN-RPA variant, corresponding to the original definition by Stephens et al. (issue 1480) and the same as the B3LYP functional in Gaussian. To restore the VWN5 definition, you can put the setting "B3LYP_WITH_VWN5 = True" in pyscf_conf.py
  warnings.warn('Since PySCF-2.3, B3LYP (and B3P86) are changed to the VWN-RPA variant, '


In [11]:
problem.hamiltonian

In [12]:
problem.num_alpha
problem.num_beta
print(problem.num_spatial_orbitals)
print(problem.num_particles)

165
(101, 100)


In [13]:
transformer = ActiveSpaceTransformer(num_electrons=2, num_spatial_orbitals=2, active_orbitals=[0,1])
#transformer.prepare_active_space(problem.num_particles, problem.num_spatial_orbitals)
#transformer = FreezeCoreTransformer(True)
#transformer.prepare_active_space(molecule, problem.num_spatial_orbitals)
redproblem= transformer.transform(problem)

In [14]:
mapper = JordanWignerMapper()
#mapper = ParityMapper(num_particles=redproblem.num_particles)

In [15]:
ansatz = UCCSD(
    num_particles = redproblem.num_particles,
    num_spatial_orbitals = problem.num_spatial_orbitals,
    qubit_mapper= mapper,
    initial_state = HartreeFock(
        num_particles = redproblem.num_particles,
        num_spatial_orbitals = redproblem.num_spatial_orbitals,
        qubit_mapper = mapper
    )
)
from qiskit_nature.second_q.algorithms.initial_points import HFInitialPoint
initial_point = HFInitialPoint()
initial_point.ansatz = ansatz
initial_point.problem = redproblem

print(ansatz.num_qubits)
print(redproblem.num_qubits)

330


AttributeError: 'ElectronicStructureProblem' object has no attribute 'num_qubits'

In [ ]:
redproblem.hamiltonian.num_qubits

AttributeError: 'ElectronicEnergy' object has no attribute 'num_qubits'

In [ ]:
%store redproblem
%store problem
%store ansatz

UsageError: Unknown variable 'redproblem'


In [25]:
redproblem2 = redproblem
%store redproblem2

Stored 'redproblem2' (ElectronicStructureProblem)


/home/codespace/.local/lib/python3.10/site-packages/IPython/extensions/storemagic.py:229: UserWarning: using autorestore/redproblem2 requires you to install the `pickleshare` library.
  db[ 'autorestore/' + arg ] = obj


In [ ]:
%store -r redproblem

TypeError: 'PickleShareDB' object is not subscriptable

In [6]:

filename = "GinsenosidesParams.txt"
def ccallback(nfev, parameters, energy, stddev):
    with open("GinsenosidesParams.txt", "a") as f:
        f.write('\n\n\n')
        f.write(parameters)
    print("hello")
solver = VQE(Estimator(), ansatz, SLSQP(), callback = ccallback)
solver.initial_point = initial_point.to_numpy_array()

NameError: name 'ansatz' is not defined

In [28]:
print(redproblem.num_particles)
print(redproblem.num_spatial_orbitals)

(2, 2)
2


In [33]:
from qiskit_nature.second_q.algorithms import GroundStateEigensolver
true_solver = GroundStateEigensolver(mapper, solver)
results = true_solver.solve(problem)
with open("Ginsenosides-Results.txt", "w") as f:
   f.write(results)

: 

In [ ]:
true_solver.solve(problem)

: 

In [ ]:
print(ansatz.num_qubits)
print(problem.num_qubits)

NameError: name 'ansatz' is not defined

In [ ]:
with open("GinsenosidesParams.txt") as f:
    print(f.read())